In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import plotly.graph_objects as go
import tensorflow as tf
from keras_nlp.layers import TransformerEncoder

Using TensorFlow backend


In [6]:
#load data
asset_pchange_list = []
asset_price_list = []
asset_name_list = []

df = pd.read_excel('Data.xlsx', sheet_name=4)
date_list = np.flip(df.iloc[:,0].to_numpy().astype('datetime64[D]'))
num_of_asset = len(df.columns)
for i in range(1, num_of_asset):
    asset_array = df.iloc[:,i].to_numpy()
    mask = ~np.isnan(asset_array)
    asset_array_no_na = asset_array[mask]
    asset_price_list.append(np.flip(asset_array_no_na))
    asset_name_list.append(df.columns[i])

p_date_list = date_list[1:]
for i in range(len(asset_price_list)):
    asset_price = asset_price_list[i]
    print(asset_name_list[i], i)
    if i < 12 or (len(asset_price_list)>20 and i<19):
    
        try:
            diff_list = np.diff(asset_price) / asset_price[:-1]
        except:
            print(i)
    else:
        diff_list = np.diff(asset_price)
    asset_pchange_list.append(diff_list)

SPX index 0


In [7]:
spx_price = np.flip(df.filter(['SPX index']).values)
date_list = date_list
spx_price

array([[1441.47],
       [1465.15],
       [1441.36],
       ...,
       [4536.34],
       [4582.23],
       [4478.03]])

In [8]:
#Train test split
training_data_len = math.ceil(len(spx_price)* .8)
training_data_len
#Scale data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(spx_price)
scaled_data

#Train test split
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []
look_back_window = 14

for i in range(look_back_window, len(train_data)):
    x_train.append(train_data[i-look_back_window:i, 0])
    y_train.append(train_data[i, 0])
    if i <=15:
        print(x_train)
        print(y_train)
#Convert to np array
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape, y_train.shape

[array([0.18567895, 0.19147889, 0.185652  , 0.16576369, 0.18149064,
       0.172367  , 0.16231753, 0.15919957, 0.17776771, 0.1743142 ,
       0.19131233, 0.20674047, 0.1996669 , 0.2040193 ])]
[0.16488194376408347]
[array([0.18567895, 0.19147889, 0.185652  , 0.16576369, 0.18149064,
       0.172367  , 0.16231753, 0.15919957, 0.17776771, 0.1743142 ,
       0.19131233, 0.20674047, 0.1996669 , 0.2040193 ]), array([0.19147889, 0.185652  , 0.16576369, 0.18149064, 0.172367  ,
       0.16231753, 0.15919957, 0.17776771, 0.1743142 , 0.19131233,
       0.20674047, 0.1996669 , 0.2040193 , 0.16488194])]
[0.16488194376408347, 0.1839815812677574]


((971, 14, 1), (971,))

In [9]:
model = Sequential()
model.add(LSTM(14, return_sequences=True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(units=14, return_sequences=False))
model.add(Dense(7))
model.add(Dense(1))
model.compile(optimizer='adam', loss="mean_squared_error")

In [10]:
transformer_model = Sequential()
transformer_model.add(TransformerEncoder(10, 4, input_shape = (x_train.shape[1], 1)))
transformer_model.add(Dense(1))
transformer_model.compile(optimizer='adam', loss="mean_squared_error")

StagingError: Exception encountered when calling layer "transformer_encoder" (type TransformerEncoder).

in user code:

    File "c:\Users\intern2\.conda\envs\machineLearnng\lib\site-packages\keras_nlp\src\layers\modeling\transformer_encoder.py", line 195, in call  *
        x = self._self_attention_layer(
    File "c:\Users\intern2\.conda\envs\machineLearnng\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\intern2\.conda\envs\machineLearnng\lib\site-packages\keras\layers\attention\multi_head_attention.py", line 521, in _compute_attention
        query = tf.multiply(query, 1.0 / math.sqrt(float(self._key_dim)))

    ZeroDivisionError: Exception encountered when calling layer "multi_head_attention" "                 f"(type MultiHeadAttention).
    
    float division by zero
    
    Call arguments received by layer "multi_head_attention" "                 f"(type MultiHeadAttention

In [91]:
model.fit(x_train, y_train, batch_size= 1, epochs=5)
test_data = scaled_data[training_data_len-look_back_window:, :]

x_test = []
y_test = spx_price[training_data_len:, :]

for i in range(look_back_window, len(test_data)):
    x_test.append(test_data[i-look_back_window:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print(rmse)
train = np.squeeze(spx_price[:training_data_len])
valid = np.squeeze(spx_price[training_data_len:])
predictions = np.squeeze(predictions)
train_date_list = date_list[:training_data_len]
valid_date_list = date_list[training_data_len:]
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list, y=predictions, name='predict',  mode='lines'))
fig.show()

LSTM()

Epoch 1/5
971/971 [==============================] - 9s 5ms/step - loss: 9.7180e-04
Epoch 2/5
971/971 [==============================] - 5s 5ms/step - loss: 3.6760e-04
Epoch 3/5
971/971 [==============================] - 5s 5ms/step - loss: 3.0427e-04
Epoch 4/5
971/971 [==============================] - 5s 6ms/step - loss: 2.4455e-04
Epoch 5/5
8/8 [==============================] - 1s 4ms/step
86.41328732374238
